In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, Column
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [2]:
    spark = SparkSession \
        .builder \
        .appName("test pipling for local") \
        .getOrCreate()

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.driver.port', '45163'),
 ('spark.app.name', 'test pipling for local'),
 ('spark.driver.host', 'edebaaa10893'),
 ('spark.app.id', 'local-1561755226042'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

### Imporing json song  data into spark

In [4]:
dfSong = spark.read.json("data/song_data/*/*/*/*.json")

In [5]:
dfSong.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [7]:
dfSong.limit(5).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARDR4AC1187FB371A1,NaN,,NaN,Montserrat Caballé;Placido Domingo;Vicente Sar...,511.16363,1,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,0
1,AREBBGV1187FB523D2,NaN,"Houston, TX",NaN,Mike Jones (Featuring CJ_ Mello & Lil' Bran),173.66159,1,SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),0
2,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
3,ARPBNLO1187FB3D52F,40.71455,"New York, NY",-74.00712,Tiny Tim,43.36281,1,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,2000
4,ARDNS031187B9924F0,32.67828,Georgia,-83.22295,Tim Wilson,186.48771,1,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,2005


### Imporing json log  data into spark

In [5]:
dfLog = spark.read.json("data/log_data/*.json")

In [9]:
dfLog.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [10]:
dfLog.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12


## Make songs table

In [7]:
dfSongTable = dfSong.select(['song_id', 'title', 'artist_id', 'year', 'duration'])

AnalysisException: "cannot resolve '`page`' given input columns: [num_songs, duration, song_id, artist_location, artist_longitude, artist_latitude, title, artist_id, year, artist_name]; line 1 pos 0;\n'Filter ('page = NextSong)\n+- Relation[artist_id#6,artist_latitude#7,artist_location#8,artist_longitude#9,artist_name#10,duration#11,num_songs#12L,song_id#13,title#14,year#15L] json\n"

In [12]:
dfSongTable.limit(5).toPandas()

,song_id,title,artist_id,year,duration
0,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,ARDR4AC1187FB371A1,0,511.16363
1,SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),AREBBGV1187FB523D2,0,173.66159
2,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,ARMAC4T1187FB3FA4C,2004,207.77751
3,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,ARPBNLO1187FB3D52F,2000,43.36281
4,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,ARDNS031187B9924F0,2005,186.48771


### create parquet song table files partitioned by year,artist.

In [12]:
dfSongTable.write.partitionBy("year", "artist_id").parquet("SongTable.parquet")

AnalysisException: 'path file:/home/workspace/SongTable.parquet already exists.;'

### Create parquet artist table table files 

In [12]:
dfArtistTable = dfSong.select(['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude'])

In [13]:
dfArtistTable.write.parquet("ArtistTable.parquet")

AnalysisException: 'path file:/home/workspace/ArtistTable.parquet already exists.;'

## Make User table

In [8]:
dfUserTable = dfLog.filter('page = "NextSong"').select(['userId', 'firstName', 'lastName', 'gender', 'level'])

In [15]:
dfUserTable.show(5)

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|     9|    Wyatt|   Scott|     M| free|
|    12|   Austin| Rosales|     M| free|
+------+---------+--------+------+-----+
only showing top 5 rows



In [23]:
dfUserTable.write.parquet("UserTable.parquet")

### Create time table

In [132]:
get_time_stamp = udf( lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y-%m-%d %H:%M:%S'))

In [133]:
dfLog = dfLog.withColumn("time_stamp", get_time_stamp(dfLog.ts))

In [121]:
dfLog.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- time_stamp: string (nullable = true)



In [146]:
dfLog.select('time_stamp', hour('time_stamp'), weekofyear('time_stamp'), month('time_stamp'), year('time_stamp'), date_format('time_stamp', 'EEEE')).show(5)

+-------------------+----------------+----------------------+-----------------+----------------+-----------------------------+
|         time_stamp|hour(time_stamp)|weekofyear(time_stamp)|month(time_stamp)|year(time_stamp)|date_format(time_stamp, EEEE)|
+-------------------+----------------+----------------------+-----------------+----------------+-----------------------------+
|2018-11-15 00:30:26|               0|                    46|               11|            2018|                     Thursday|
|2018-11-15 00:41:21|               0|                    46|               11|            2018|                     Thursday|
|2018-11-15 00:45:41|               0|                    46|               11|            2018|                     Thursday|
|2018-11-15 01:57:51|               1|                    46|               11|            2018|                     Thursday|
|2018-11-15 03:29:37|               3|                    46|               11|            2018|               

In [153]:
time_table = dfLog.select('ts', hour('time_stamp'), dayofmonth('time_stamp'), weekofyear('time_stamp'), month('time_stamp'), year('time_stamp'), date_format('time_stamp', 'EEEE'))

In [154]:
time_table.show(5)

+-------------+----------------+----------------------+----------------------+-----------------+----------------+-----------------------------+
|           ts|hour(time_stamp)|dayofmonth(time_stamp)|weekofyear(time_stamp)|month(time_stamp)|year(time_stamp)|date_format(time_stamp, EEEE)|
+-------------+----------------+----------------------+----------------------+-----------------+----------------+-----------------------------+
|1542241826796|               0|                    15|                    46|               11|            2018|                     Thursday|
|1542242481796|               0|                    15|                    46|               11|            2018|                     Thursday|
|1542242741796|               0|                    15|                    46|               11|            2018|                     Thursday|
|1542247071796|               1|                    15|                    46|               11|            2018|                     Th

### Make the song paly table

In [14]:
artist_and_song = dfSong.select('song_id', 'artist_id', 'title', 'artist_name', 'duration')

In [15]:
artist_and_song.show(5)

+------------------+------------------+--------------------+--------------------+---------+
|           song_id|         artist_id|               title|         artist_name| duration|
+------------------+------------------+--------------------+--------------------+---------+
|SOBAYLL12A8C138AF9|ARDR4AC1187FB371A1|Sono andati? Fing...|Montserrat Caball...|511.16363|
|SOOLYAZ12A6701F4A6|AREBBGV1187FB523D2|Laws Patrolling (...|Mike Jones (Featu...|173.66159|
|SOBBUGU12A8C13E95D|ARMAC4T1187FB3FA4C|Setting Fire to S...|The Dillinger Esc...|207.77751|
|SOAOIBZ12AB01815BE|ARPBNLO1187FB3D52F|I Hold Your Hand ...|            Tiny Tim| 43.36281|
|SONYPOM12A8C13B2D7|ARDNS031187B9924F0|I Think My Wife I...|          Tim Wilson|186.48771|
+------------------+------------------+--------------------+--------------------+---------+
only showing top 5 rows



In [9]:
df_paysong_table = artist_and_song.join(dfLog).where((artist_and_song.title ==  dfLog.song) & (artist_and_song.artist_name == dfLog.artist) & (artist_and_song.duration == dfLog.length)).select('ts', 'userid', 'level', 'song_id', 'artist_id', 'sessionid', 'location', 'useragent') 

NameError: name 'artist_and_song' is not defined

In [38]:
df_paysong_table.show(5)

+-------------+------+-----+------------------+------------------+---------+--------------------+--------------------+
|           ts|userid|level|           song_id|         artist_id|sessionid|            location|           useragent|
+-------------+------+-----+------------------+------------------+---------+--------------------+--------------------+
|1542837407796|    15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|      818|Chicago-Napervill...|"Mozilla/5.0 (X11...|
+-------------+------+-----+------------------+------------------+---------+--------------------+--------------------+

